In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## گام ۱: تعریف مدل‌های پایدانتیک برای خروجی‌های ساختارمند

این مدل‌ها **اسکیمایی** را که عوامل برمی‌گردانند تعریف می‌کنند. استفاده از `response_format` همراه با پایدانتیک تضمین می‌کند:
- ✅ استخراج داده‌های نوع ایمن
- ✅ اعتبارسنجی خودکار
- ✅ بدون خطاهای پارسینگ از پاسخ‌های متنی آزاد
- ✅ مسیردهی شرطی آسان بر اساس فیلدها


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## گام ۲: ایجاد ابزار رزرو هتل

این ابزار چیزی است که **availability_agent** برای بررسی در دسترس بودن اتاق‌ها صدا خواهد زد. ما از دکوراتور `@ai_function` برای:
- تبدیل یک تابع پایتون به ابزاری قابل فراخوانی توسط هوش مصنوعی
- تولید خودکار طرح JSON برای مدل زبانی بزرگ (LLM)
- مدیریت اعتبارسنجی پارامترها
- فعال‌سازی فراخوانی خودکار توسط عوامل

برای این نمایش:
- **استکهلم، سیاتل، توکیو، لندن، آمستردام** → اتاق دارند ✅
- **سایر شهرها** → اتاق ندارند ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## مرحله ۳: تعریف توابع شرطی برای تعیین مسیر

این توابع پاسخ عامل را بررسی کرده و مشخص می‌کنند کدام مسیر در جریان کار باید طی شود.

**الگوی کلیدی:**
۱. بررسی اینکه آیا پیام مربوط به `AgentExecutorResponse` است  
۲. تجزیه خروجی ساختاریافته (مدل Pydantic)  
۳. بازگرداندن `True` یا `False` برای کنترل مسیر

در جریان کار، این شرایط روی **یال‌ها** ارزیابی می‌شوند تا تصمیم بگیرند کدام اجراکننده بعدی فراخوانی شود.


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## گام ۴: ایجاد اجرای سفارشی نمایش

اجراکننده‌ها اجزای جریان کاری هستند که تبدیل‌ها یا اثرات جانبی را انجام می‌دهند. ما از دکوریتور `@executor` برای ایجاد یک اجرای سفارشی استفاده می‌کنیم که نتیجه نهایی را نمایش می‌دهد.

**مفاهیم کلیدی:**
- `@executor(id="...")` - ثبت یک تابع به عنوان اجرای جریان کاری
- `WorkflowContext[Never, str]` - نوع‌های راهنمای ورودی/خروجی
- `ctx.yield_output(...)` - بازگرداندن نتیجه نهایی جریان کاری


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## مرحله ۵: بارگذاری متغیرهای محیطی

کلاینت LLM را پیکربندی کنید. این مثال با موارد زیر کار می‌کند:
- **مدل‌های GitHub** (سطح رایگان با توکن GitHub)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## مرحله ۶: ایجاد عامل‌های هوش مصنوعی با خروجی‌های ساختاریافته

ما **سه عامل تخصصی** ایجاد می‌کنیم که هر کدام در یک `AgentExecutor` بسته‌بندی شده‌اند:

۱. **availability_agent** - بررسی دسترسی هتل با استفاده از ابزار
۲. **alternative_agent** - پیشنهاد شهرهای جایگزین (زمانی که اتاقی نیست)
۳. **booking_agent** - تشویق به رزرو (زمانی که اتاق موجود است)

**ویژگی‌های کلیدی:**
- `tools=[hotel_booking]` - ارائه ابزار به عامل
- `response_format=PydanticModel` - اجباری کردن خروجی JSON ساختاریافته
- `AgentExecutor(..., id="...")` - بسته‌بندی عامل برای استفاده در جریان کاری


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## مرحله ۷: ساخت جریان کاری با لبه‌های شرطی

اکنون از `WorkflowBuilder` برای ساخت گراف با مسیرهای شرطی استفاده می‌کنیم:

**ساختار جریان کاری:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**متدهای کلیدی:**
- `.set_start_executor(...)` - تعیین نقطه ورود
- `.add_edge(from, to, condition=...)` - اضافه کردن لبه شرطی
- `.build()` - نهایی کردن جریان کاری


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## گام ۸: اجرای تست مورد ۱ - شهری بدون ظرفیت (پاریس)

بیایید مسیر **بدون ظرفیت** را با درخواست هتل‌ها در پاریس (که در شبیه‌سازی ما اتاقی ندارد) آزمایش کنیم.


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## گام ۹: اجرای مورد آزمایشی ۲ - شهر با موجودی (استکهلم)

اکنون بیایید مسیر **موجودی** را با درخواست هتل‌ها در استکهلم (که در شبیه‌سازی ما اتاق‌هایی دارد) آزمایش کنیم.


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## نکات کلیدی و گام‌های بعدی

### ✅ آنچه یاد گرفته‌اید:

1. **الگوی WorkflowBuilder**
   - استفاده از `.set_start_executor()` برای تعریف نقطه ورودی
   - استفاده از `.add_edge(from, to, condition=...)` برای مسیرهای شرطی
   - فراخوانی `.build()` برای نهایی کردن جریان کاری

2. **مسیر‌دهی شرطی**
   - توابع شرطی پاسخ‌های `AgentExecutorResponse` را بررسی می‌کنند
   - خروجی‌های ساختار یافته را برای تصمیم‌گیری مسیرها تجزیه می‌کنند
   - بازگرداندن `True` برای فعال کردن یک مسیر، `False` برای رد کردن آن

3. **ادغام ابزارها**
   - استفاده از `@ai_function` برای تبدیل توابع پایتون به ابزارهای هوش مصنوعی
   - عوامل به‌صورت خودکار هنگام نیاز به ابزارها فراخوانی می‌کنند
   - ابزارها JSON بازمی‌گردانند که عوامل می‌توانند تجزیه کنند

4. **خروجی‌های ساختار یافته**
   - استفاده از مدل‌های Pydantic برای استخراج داده‌های نوع ایمن
   - تنظیم `response_format=MyModel` هنگام ایجاد عوامل
   - تجزیه پاسخ‌ها با `Model.model_validate_json()`

5. **اجراکننده‌های سفارشی**
   - استفاده از `@executor(id="...")` برای ایجاد اجزای جریان کاری
   - اجراکننده‌ها می‌توانند داده‌ها را تبدیل کنند یا اثرات جانبی داشته باشند
   - استفاده از `ctx.yield_output()` برای تولید نتایج جریان کاری

### 🚀 کاربردهای واقعی:

- **رزرو سفر**: بررسی موجودی، پیشنهاد جایگزین‌ها، مقایسه گزینه‌ها
- **خدمات مشتریان**: مسیر‌دهی بر اساس نوع مشکل، احساسات، اولویت
- **تجارت الکترونیک**: بررسی موجودی، پیشنهاد جایگزین‌ها، پردازش سفارش‌ها
- **نظارت بر محتوا**: مسیر‌دهی بر اساس امتیازهای سمی بودن، گزارش‌های کاربران
- **جریان‌های تصویب**: مسیر‌دهی بر اساس مبلغ، نقش کاربر، سطح ریسک
- **پردازش چندمرحله‌ای**: مسیر‌دهی بر اساس کیفیت داده‌ها، کامل بودن

### 📚 گام‌های بعدی:

- افزودن شرایط پیچیده‌تر (چندین معیار)
- پیاده‌سازی حلقه‌ها با مدیریت وضعیت جریان کاری
- افزودن جریان‌های فرعی برای اجزای قابل استفاده مجدد
- ادغام با APIهای واقعی (رزرو هتل، سیستم‌های موجودی)
- افزودن مدیریت خطا و مسیرهای جایگزین
- مصورسازی جریان‌های کاری با ابزارهای مصور داخلی


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. اگرچه ما برای دقت تلاش می‌کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است حاوی اشتباهات یا نادرستی‌هایی باشند. سند اصلی به زبان بومی خود باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حیاتی، ترجمه حرفه‌ای انسانی توصیه می‌شود. ما مسئول هیچگونه سوءتفاهم یا برداشت نادرست ناشی از استفاده از این ترجمه نیستیم.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
